<a href="https://colab.research.google.com/github/Vijeet24/AI-Music-Generation/blob/main/GANS_for_Music_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from music21 import converter , instrument , chord ,note , stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils_utils
midi = converter.sparse( 'location.mid')
midi
midi.show('midi')
midi.show('text')
element_to_parse = midi.flat.notes
len(element_to_parse)
for e in element_to_parse:
    print(e,e.offset)
notes_demo = []

for ele in element_to_parse:
    if isinstance(ele,notes.Notes):
        notes_demo.append(str(ele.pitch))
    
    elif isinstance(ele,chord.Chord):
        notes_demo.append("+".join(str(n) for n in ele.normaloOrder))

Processing all files


In [ ]:
notes = []


for files in glob.glob('/midisongs/.midi'):
    midi = converter.parse(file)
    
    print('parsing is%'%file)
    
    element_to_parse = midi.flat.notes
    
    
    for ele in element_to_parse:
        if isinstance(ele,notes.Notes):
            notes_demo.append(str(ele.pitch))

        elif isinstance(ele,chord.Chord):
             notes_demo.append("+".join(str(n) for n in ele.normaloOrder))

   

In [ ]:
with open('notes','wb') as filepath:
    pickle.dump(notes,filepath)
with open('notes','rb') as f:
    notes = pickle.load(f)
n_vocab = len(set(notes))
print("total notes - ",len(notes))
print("unique notes - ",n_vocab)

Prepare sequential data


In [ ]:
sequence_length = 100
pitch_name = sorted(set(notes))
ele_to_int = dict((ele,num) for ele ,num in enumerate(pitch_name)
network_input  = []
network_output = []
for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i + sequence_length]
    seq_out = notes[i + sequence_length]
    
    network_input.append([ele_to_int[ch] for ch in seq_in])
    network_output.append(ele_to_int[seq_out])
network_input
n_pattern  = len(network_input)
print(n_pattern)
network_input = np.reshape(network_input,(n_pattern,sequence_length,1))
print(network_input.shape)

In [ ]:
normalized_network__input = network_input/(n_vocab)
network_output = np.utils.to_categorical(network_output)
print(normalized_network_input.shape)
print(network_output.shape)

Generator fake samples


In [ ]:
def generate_fake_samples(n):
	# generate inputs in [-1, 1]
	X1 = -1 + rand(n) * 2
	# generate outputs in [-1, 1]
	X2 = -1 + rand(n) * 2
	# stack arrays
	X1 = X1.reshape(n, 1)
	X2 = X2.reshape(n, 1)
	X = hstack((X1, X2))
	# generate class labels
	y = zeros((n, 1))
	return X, y

Discriminator Function


In [ ]:

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

# define the standalone discriminator model
def define_discriminator(n_inputs=2):
	model = Sequential()
	model.add(Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# define the discriminator model
model = define_discriminator()
# summarize the model

Generator Function

In [ ]:

# define the standalone generator model
def define_generator(latent_dim, n_outputs=2):
	model = Sequential()
	model.add(Dense(15, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim))
	model.add(Dense(n_outputs, activation='linear'))
	return model

Input data

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
	# generate points in the latent space
	x_input = randn(latent_dim * n)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n, latent_dim)
	return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n, 1))
	return X, y

Evaluate the discriminator

In [ ]:
evaluate the discriminator and plot real and fake points
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
	# prepare real samples
	x_real, y_real = generate_real_samples(n)
	# evaluate discriminator on real examples
	_, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
	# evaluate discriminator on fake examples
	_, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print(epoch, acc_real, acc_fake)
	# scatter plot real and fake data points
	pyplot.scatter(x_real[:, 0], x_real[:, 1], color='red')
	pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
	pyplot.show()

Train the generator and discriminator

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=128, n_eval=2000):
	# determine half the size of one batch, for updating the discriminator
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# prepare real samples
		x_real, y_real = generate_real_samples(half_batch)
		# prepare fake examples
		x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator
		d_model.train_on_batch(x_real, y_real)
		d_model.train_on_batch(x_fake, y_fake)
		# prepare points in latent space as input for the generator
		x_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		gan_model.train_on_batch(x_gan, y_gan)
		# evaluate the model every n_eval epochs
		if (i+1) % n_eval == 0:
			summarize_performance(i, g_model, d_model, latent_dim)
 
# size of the latent space
latent_dim = 5
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)

Create MIDI File

In [ ]:
offset = 0
output_note = []

for pattern in prediction_output:
    
    if(' + ' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        
    for current_notes in notes_in_chords:
        new_note = notes.Notes(int(current_note))
        new_notes.storedInstrument = instrument.Piano()
        temp_notes.append(new_notes)
        
    new_chord = chord.Chord(temp_notes)
    new_chord.offset = offset 
    output_notes.append(new_chord)
    
    
    else:
        new_notes = notes.Notes(prediction_output[4])
        new_notes.offset = offset
        new_notes.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
        
    offset += 0.5


midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = 'test_output.mid')
midi_stream.show('midi')